In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Eczema"
cohort = "GSE57225"

# Input paths
in_trait_dir = "../../input/GEO/Eczema"
in_cohort_dir = "../../input/GEO/Eczema/GSE57225"

# Output paths
out_data_file = "../../output/preprocess/Eczema/GSE57225.csv"
out_gene_data_file = "../../output/preprocess/Eczema/gene_data/GSE57225.csv"
out_clinical_data_file = "../../output/preprocess/Eczema/clinical_data/GSE57225.csv"
json_path = "../../output/preprocess/Eczema/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Intra-individual genome expression analysis reveals a specific molecular signature of psoriasis and eczema"
!Series_summary	"Comparing molecular signatures of psoriasis and eczema in patients co-affected by both diseases provides a comprehensive understanding of disease pathogenesis as well as a diagnostic tool to differentiate these widespread inflammatory skin diseases."
!Series_overall_design	"In patients affected by both psoriasis and non-atopic or atopic eczema simultaneously (n=24), whole genome expression arrays of psoriasis, eczema, and non-involved skin were performed"
!Series_overall_design	"Arrays MQ_35 and MQ_41 did not pass quality control and thus were not normalized and were excluded from this Series."
Sample Characteristics Dictionary:
{0: ['individual: EP', 'individual: KK', 'individual: KP', 'individual: SI', 'individual: MM', 'individual: ZA', 'individual: LA', 'individual: MOF', 'individual: Mattes', 'individual: SS2', 'individ

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Dict, Any, Callable, List
import re

# Display sample characteristics to understand the dataset
sample_characteristics = {0: ['individual: EP', 'individual: KK', 'individual: KP', 'individual: SI', 'individual: MM', 'individual: ZA', 'individual: LA', 'individual: MOF', 'individual: Mattes', 'individual: SS2', 'individual: Klose', 'individual: JB', 'individual: LI', 'individual: HI', 'individual: SM', 'individual: BC', 'individual: BS (31y)', 'individual: VI', 'individual: BA', 'individual: BS (33y)', 'individual: SP', 'individual: MPG', 'individual: WP'], 1: ['disease state: psoriasis', 'disease state: eczema', 'disease state: control (non-involved)'], 2: ['tissue: skin'], 3: ['gender: male', 'gender: female'], 4: ['age: 48y', 'age: 40y', 'age: 65y', 'age: 35y', 'age: 27y', 'age: 72y', 'age: 33y', 'age: 58y', 'age: 56y', 'age: 46y', 'age: 55y', 'age: 53y', 'age: 31y', 'age: 42y', 'age: 43y', 'age: 20y', 'age: 41y']}

print("Review of Sample Characteristics Dictionary:")
for key, values in sample_characteristics.items():
    print(f"Key {key}: {values}")

# 1. Gene Expression Data Availability
# Based on the summary and title, this dataset appears to contain genome expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary:
# Key 1 contains disease state which can be used for our trait (Eczema)
# Key 4 contains age information
# Key 3 contains gender information
trait_row = 1  # 'disease state: psoriasis', 'disease state: eczema', 'disease state: control (non-involved)'
age_row = 4    # Age information is available
gender_row = 3  # Gender information is available

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """
    Convert trait information to binary format.
    1 for eczema, 0 for control, None for psoriasis (as we're focusing on eczema)
    """
    if value is None:
        return None
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'eczema' in value.lower():
        return 1  # Case
    elif 'control' in value.lower() or 'non-involved' in value.lower():
        return 0  # Control
    else:
        return None  # Other conditions like psoriasis

def convert_age(value: str) -> Optional[float]:
    """
    Convert age information to continuous format.
    Extract numeric age value from strings like 'age: 48y'
    """
    if value is None:
        return None
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Use regex to extract numbers from the age string
    age_match = re.search(r'(\d+)', value)
    if age_match:
        return float(age_match.group(1))
    return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender information to binary format.
    0 for female, 1 for male
    """
    if value is None:
        return None
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    return None

# 3. Save Metadata
# Initial filtering on usability - trait data is available if trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Print a message about clinical data extraction
print("Note: We have identified that trait, age, and gender data are available in the sample characteristics.")
print(f"trait_row = {trait_row}, age_row = {age_row}, gender_row = {gender_row}")
print("However, since we don't have access to the actual clinical data file, we'll skip the data extraction step for now.")
print("The sample characteristics provided are metadata about the unique values, not the actual clinical data.")


Review of Sample Characteristics Dictionary:
Key 0: ['individual: EP', 'individual: KK', 'individual: KP', 'individual: SI', 'individual: MM', 'individual: ZA', 'individual: LA', 'individual: MOF', 'individual: Mattes', 'individual: SS2', 'individual: Klose', 'individual: JB', 'individual: LI', 'individual: HI', 'individual: SM', 'individual: BC', 'individual: BS (31y)', 'individual: VI', 'individual: BA', 'individual: BS (33y)', 'individual: SP', 'individual: MPG', 'individual: WP']
Key 1: ['disease state: psoriasis', 'disease state: eczema', 'disease state: control (non-involved)']
Key 2: ['tissue: skin']
Key 3: ['gender: male', 'gender: female']
Key 4: ['age: 48y', 'age: 40y', 'age: 65y', 'age: 35y', 'age: 27y', 'age: 72y', 'age: 33y', 'age: 58y', 'age: 56y', 'age: 46y', 'age: 55y', 'age: 53y', 'age: 31y', 'age: 42y', 'age: 43y', 'age: 20y', 'age: 41y']
Note: We have identified that trait, age, and gender data are available in the sample characteristics.
trait_row = 1, age_row = 4, 

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Eczema/GSE57225/GSE57225_series_matrix.txt.gz


Gene data shape: (42044, 62)
First 20 gene/probe identifiers:
Index(['A_19_P00315452', 'A_19_P00315459', 'A_19_P00315469', 'A_19_P00315473',
       'A_19_P00315482', 'A_19_P00315490', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315499', 'A_19_P00315502', 'A_19_P00315504', 'A_19_P00315506',
       'A_19_P00315508', 'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315523',
       'A_19_P00315524', 'A_19_P00315526', 'A_19_P00315527', 'A_19_P00315528'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# These identifiers appear to be Agilent microarray probe IDs (starting with A_19_P),
# not standard human gene symbols (like ACTB, TP53, etc.)
# Therefore, mapping to gene symbols will be required

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation contain probe IDs and gene symbols
# Based on the gene annotation preview, the relevant columns are:
# - 'ID': contains probe IDs like those in gene_data.index (A_19_P...)
# - 'GENE_SYMBOL': contains gene symbols

# 2. Get a gene mapping dataframe with the two relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print(f"Generated gene mapping dataframe with shape: {gene_mapping.shape}")
print("Sample of gene mapping data:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted to gene-level expression data with shape: {gene_data.shape}")
print("Sample of gene expression data:")
print(gene_data.head())

# Save the gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Generated gene mapping dataframe with shape: (30436, 2)
Sample of gene mapping data:
                ID         Gene
21  A_19_P00315502  XLOC_008373
25  A_19_P00315518  XLOC_005810
26  A_19_P00315519  XLOC_004914
28  A_19_P00315524  XLOC_014192
31  A_19_P00315528  XLOC_008370


Converted to gene-level expression data with shape: (19725, 62)
Sample of gene expression data:
          GSM1377461  GSM1377462  GSM1377463  GSM1377464  GSM1377465  \
Gene                                                                   
A1BG       12.977107   13.381272   11.371641   13.209966   12.411935   
A1BG-AS1    6.935017    7.078788    7.111287    7.637369    7.016705   
A1CF        4.846280    5.042712    4.946058    4.353776    5.875778   
A2LD1       8.322505    8.170192    8.379592    8.217080    7.549794   
A2M        11.275092   12.710169   12.025984   12.847757   12.482553   

          GSM1377466  GSM1377467  GSM1377468  GSM1377469  GSM1377470  ...  \
Gene                                                                  ...   
A1BG       13.526912   12.966083   12.651566   13.259562   12.608978  ...   
A1BG-AS1    7.483763    7.429407    6.585663    6.524840    6.261774  ...   
A1CF        4.998552    5.307923    4.893777    4.880007    4.513105  ...   
A2LD1       8.

Gene expression data saved to: ../../output/preprocess/Eczema/gene_data/GSE57225.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check first if we need to complete the clinical feature extraction from Step 2
print("Checking if clinical data extraction is needed...")
if not os.path.exists(out_clinical_data_file):
    print("Clinical data file not found. Extracting clinical features from original data...")
    # Get the matrix file path
    _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get the clinical data from the matrix file
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    _, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Define conversion functions from Step 2
    def convert_trait(value: str) -> Optional[int]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'eczema' in value.lower():
            return 1  # Case
        elif 'control' in value.lower() or 'non-involved' in value.lower():
            return 0  # Control
        else:
            return None  # Other conditions like psoriasis

    def convert_age(value: str) -> Optional[float]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        age_match = re.search(r'(\d+)', value)
        if age_match:
            return float(age_match.group(1))
        return None

    def convert_gender(value: str) -> Optional[int]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'female' in value.lower():
            return 0
        elif 'male' in value.lower():
            return 1
        return None
    
    # Extract clinical features with identified rows from Step 2
    trait_row = 1
    age_row = 4
    gender_row = 3
    
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features extracted and saved to: {out_clinical_data_file}")
else:
    print(f"Clinical data file already exists at: {out_clinical_data_file}")
    clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)

# Now proceed with Step 7 as originally planned
# 1. Normalize gene symbols using NCBI Gene database information
print("\nNormalizing gene symbols...")
try:
    # Load the gene data if needed
    if 'gene_data' not in locals() or gene_data is None:
        gene_data = pd.read_csv(out_gene_data_file, index_col=0)
        
    # Normalize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"Sample of normalized gene symbols: {normalized_gene_data.index[:10].tolist()}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
        print(linked_data.iloc[:5, :5])
    else:
        print(linked_data)
    
    # 4. Handle missing values
    print("\nHandling missing values...")
    linked_data_clean = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
    
    # 5. Check for bias in the dataset
    print("\nChecking for bias in dataset features...")
    is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)
    
    # 6. Conduct final quality validation
    note = "Dataset contains gene expression data from skin biopsies comparing different skin conditions including eczema (atopic dermatitis and contact eczema) against other conditions like psoriasis and healthy controls."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data_clean,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data_clean.to_csv(out_data_file, index=True)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for associative studies. Linked data not saved.")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # If processing fails, we should still validate the dataset status
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,  # We know trait data is available from step 2
        is_biased=True,  # Set to True to ensure it's not marked usable
        df=pd.DataFrame(),  # Empty dataframe since processing failed
        note=f"Failed to process data: {e}"
    )
    print("Dataset validation completed with error status.")

Checking if clinical data extraction is needed...
Clinical data file not found. Extracting clinical features from original data...
Clinical features extracted and saved to: ../../output/preprocess/Eczema/clinical_data/GSE57225.csv

Normalizing gene symbols...


Gene data shape after normalization: (19361, 62)
Sample of normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Eczema/gene_data/GSE57225.csv

Linking clinical and genetic data...
Linked data shape: (62, 19364)
Linked data preview (first 5 rows, 5 columns):
            Eczema   Age  Gender       A1BG  A1BG-AS1
GSM1377461     NaN  48.0     1.0  12.977107  6.935017
GSM1377462     1.0  48.0     1.0  13.381272  7.078788
GSM1377463     NaN  40.0     1.0  11.371641  7.111287
GSM1377464     1.0  40.0     1.0  13.209966  7.637369
GSM1377465     0.0  65.0     1.0  12.411935  7.016705

Handling missing values...


Linked data shape after handling missing values: (40, 19364)

Checking for bias in dataset features...
For the feature 'Eczema', the least common label is '0.0' with 17 occurrences. This represents 42.50% of the dataset.
The distribution of the feature 'Eczema' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.5
  50% (Median): 46.0
  75%: 56.0
Min: 20.0
Max: 72.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 32.50% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Eczema/GSE57225.csv
